In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feature-selection-data/feature_selection_data
/kaggle/input/feature-selection-data1/feature_selection_data (1)
/kaggle/input/flight-data/data_after_eda_Bivariate_and_multivariate_analysis


In [79]:
## import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [80]:
df=pd.read_csv('/kaggle/input/flight-data/data_after_eda_Bivariate_and_multivariate_analysis')
df.head()

,flight_month,is_hoiliday,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price,Stops,price_category
0,6,0,0,SpiceJet,economy,Delhi,1135,Mumbai,1265,130,6013,0,Medium
1,6,0,0,SpiceJet,economy,Delhi,380,Mumbai,520,140,6013,0,Medium
2,6,0,0,AirAsia,economy,Delhi,265,Mumbai,395,130,6016,0,Medium
3,6,0,0,Vistara,economy,Delhi,620,Mumbai,755,135,6015,0,Medium
4,6,0,0,Vistara,economy,Delhi,530,Mumbai,670,140,6015,0,Medium


In [81]:
df.shape

(298133, 13)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298133 entries, 0 to 298132
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   flight_month    298133 non-null  int64 
 1   is_hoiliday     298133 non-null  int64 
 2   day_of_week     298133 non-null  int64 
 3   airline         298133 non-null  object
 4   class           298133 non-null  object
 5   departure_city  298133 non-null  object
 6   departure_time  298133 non-null  int64 
 7   arrival_city    298133 non-null  object
 8   arrival_time    298133 non-null  int64 
 9   duration        298133 non-null  int64 
 10  price           298133 non-null  int64 
 11  Stops           298133 non-null  int64 
 12  price_category  298133 non-null  object
dtypes: int64(8), object(5)
memory usage: 29.6+ MB


In [83]:
data=df.copy()

In [84]:
category = df.select_dtypes('O').columns

In [85]:
custom_mappings = {
    'class': [['economy', 'business']]  # This will encode 'economy' as 0, 'business' as 1
}

for col in category:
    if col in custom_mappings:
        encoder = OrdinalEncoder(categories=custom_mappings[col])
    else:
        encoder = OrdinalEncoder()
    data[col] = encoder.fit_transform(data[[col]])
    print(f"{col}: {encoder.categories_}")

airline: [array(['Air India', 'AirAsia', 'GO FIRST', 'Indigo', 'SpiceJet',
       'StarAir', 'Trujet', 'Vistara'], dtype=object)]
class: [array(['economy', 'business'], dtype=object)]
departure_city: [array(['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai'],
      dtype=object)]
arrival_city: [array(['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai'],
      dtype=object)]
price_category: [array(['High', 'Low', 'Luxury', 'Medium', 'Very High'], dtype=object)]


In [86]:
X = data.drop(['price', 'price_category'], axis=1)
y = data['price']

In [77]:
y

0          6013
1          6013
2          6016
3          6015
4          6015
          ...  
298128    70304
298129    78262
298130    80285
298131    82809
298132    82809
Name: price, Length: 298133, dtype: int64

In [45]:
import shap
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Sample the dataset to speed up computation
sample_X = X.sample(n=50000, random_state=42)
sample_y = y.loc[sample_X.index]

# Train a LightGBM model with GPU support
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    device='gpu',          # 🔥 GPU acceleration
    gpu_use_dp=False,
    random_state=42
)
lgb_model.fit(sample_X, sample_y)

# SHAP TreeExplainer
explainer = shap.Explainer(lgb_model)
shap_value = explainer(sample_X)

In [46]:
# Calculate mean absolute SHAP values
feature_imp = np.abs(shap_value.values).mean(axis=0)

In [47]:
normalized_importance = feature_imp / feature_imp.sum()

# Create the SHAP feature importance DataFrame
F8_df = pd.DataFrame({
    'columns': sample_X.columns,
    'shap_rfe': normalized_importance
}).sort_values(by='shap_rfe', ascending=False)

F8_df

,columns,shap_rfe
4,class,0.729172
9,duration,0.075736
3,airline,0.063964
0,flight_month,0.030546
7,arrival_city,0.023189
5,departure_city,0.022456
8,arrival_time,0.018299
6,departure_time,0.016222
2,day_of_week,0.010381
10,Stops,0.008351


In [56]:
feature_selection_data=pd.read_csv('/kaggle/input/feature-selection-data1/feature_selection_data (1)')
feature_selection_data=feature_selection_data.rename(columns={'0':'feature_imp'})
feature_selection_data

,columns,feature_imp
0,class,0.891090
1,price_category,0.070244
2,duration,0.013902
3,arrival_city,0.004764
4,departure_city,0.004578
5,airline,0.004565
6,departure_time,0.003310
7,arrival_time,0.003099
8,flight_month,0.001862
9,day_of_week,0.001450


In [59]:
final_df=feature_selection_data.merge(F8_df, on='columns')
final_df

,columns,feature_imp,shap_rfe
0,class,0.891090,0.729172
1,duration,0.013902,0.075736
2,arrival_city,0.004764,0.023189
3,departure_city,0.004578,0.022456
4,airline,0.004565,0.063964
5,departure_time,0.003310,0.016222
6,arrival_time,0.003099,0.018299
7,flight_month,0.001862,0.030546
8,day_of_week,0.001450,0.010381
9,Stops,0.001029,0.008351


In [62]:
final_df['avg_score'] = final_df[['feature_imp', 'shap_rfe']].mean(axis=1)
final_df

,columns,feature_imp,shap_rfe,avg_score
0,class,0.891090,0.729172,0.810131
1,duration,0.013902,0.075736,0.044819
2,arrival_city,0.004764,0.023189,0.013976
3,departure_city,0.004578,0.022456,0.013517
4,airline,0.004565,0.063964,0.034265
5,departure_time,0.003310,0.016222,0.009766
6,arrival_time,0.003099,0.018299,0.010699
7,flight_month,0.001862,0.030546,0.016204
8,day_of_week,0.001450,0.010381,0.005916
9,Stops,0.001029,0.008351,0.004690


In [40]:
X.columns

Index(['flight_month', 'is_hoiliday', 'day_of_week', 'airline', 'class',
       'departure_city', 'departure_time', 'arrival_city', 'arrival_time',
       'duration', 'Stops'],
      dtype='object')

In [38]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
import numpy as np
import lightgbm as lgb
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# GPU-accelerated LightGBM model
model = lgb.LGBMRegressor(
    n_estimators=100,
    device='gpu',
    gpu_use_dp=False,
    random_state=42,verbose=-1
)

# Train the model
model.fit(X_train, y_train)

# KFold for training set
kfold1 = KFold(n_splits=10, shuffle=True, random_state=42)
cv_mae_train = cross_val_score(model, X_train, y_train, cv=kfold1, scoring='r2')

# KFold for testing set
kfold2 = KFold(n_splits=10, shuffle=True, random_state=42)
cv_mae_test = cross_val_score(model, X_test, y_test, cv=kfold2, scoring='r2')

In [39]:
print("Training MAE:", np.mean(cv_mae_train))  # Convert negative to positive
print("Testing MAE:", np.mean(cv_mae_test))

Training MAE: 0.9717851767315832
Testing MAE: 0.9707212211309656


In [20]:
X.columns

Index(['flight_month', 'is_hoiliday', 'day_of_week', 'airline', 'class',
       'departure_city', 'departure_time', 'arrival_city', 'arrival_time',
       'duration', 'Stops', 'price_category'],
      dtype='object')

In [63]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X.drop(columns=['is_hoiliday','Stops']), y, test_size=0.2, random_state=42)

# GPU-accelerated LightGBM model
model = lgb.LGBMRegressor(
    n_estimators=100,
    device='gpu',
    gpu_use_dp=False,
    random_state=42, verbosity=-1 
)

# Train the model
model.fit(X_train, y_train)

# KFold for training set
kfold1 = KFold(n_splits=10, shuffle=True, random_state=42)
cv_mae_train = cross_val_score(model, X_train, y_train, cv=kfold1, scoring='r2')

# KFold for testing set
kfold2 = KFold(n_splits=10, shuffle=True, random_state=42)
cv_mae_test = cross_val_score(model, X_test, y_test, cv=kfold2, scoring='r2')

In [64]:
print("Training MAE:", np.mean(cv_mae_train)) 
print("Testing MAE:", np.mean(cv_mae_test))

Training MAE: 0.9708980232705053
Testing MAE: 0.970011468124414


## `we can drop is_holiday and stops as they are not significant in model building `

In [87]:
export_df=X.drop(columns=['Stops','is_hoiliday'])

In [89]:
export_df['price']=y

In [90]:
export_df

,flight_month,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price
0,6,0,4.0,0.0,2.0,1135,5.0,1265,130,6013
1,6,0,4.0,0.0,2.0,380,5.0,520,140,6013
2,6,0,1.0,0.0,2.0,265,5.0,395,130,6016
3,6,0,7.0,0.0,2.0,620,5.0,755,135,6015
4,6,0,7.0,0.0,2.0,530,5.0,670,140,6015
...,...,...,...,...,...,...,...,...,...,...
298128,8,6,7.0,1.0,1.0,585,3.0,1190,605,70304
298129,8,6,7.0,1.0,1.0,750,3.0,1375,625,78262
298130,8,6,7.0,1.0,1.0,425,3.0,1255,830,80285
298131,8,6,7.0,1.0,1.0,420,3.0,1020,600,82809


In [92]:
export_df.to_csv('flight_data_after_feature_selection',index=False)